# Newspaper Segmentation

#### This scipt is recommended to be used for both pre-processing and post-processing methods

Following functions and methods are implemented in order to produce segmentation within the newspapers and further improvements will be added in future.

Thresholding(Otsu's, Basic Segmentation)
Denoising
Dilation
Run Length Smoothing Algorithm

In [75]:
import cv2
import re
import os
import numpy as np
import math
import copy
from pythonRLSA import rlsa

In [30]:
def read_all_images(filepath:str, mode:str, write=(False,False)):
    files = sorted(os.listdir(filepath), key=lambda f: int(re.sub('\D', '', f)))
    print("Total_nos_images ==>",len(files))
    
    if mode == 'binary':
        data = [cv2.imread(filepath+name, cv2.IMREAD_GRAYSCALE) for name in files]
    else:
        data = [cv2.imread(filepath+name) for name in files]
        
    if write[0]:
        path = write[1]
        if not os.path.exists(path):
            os.makedirs(path)
        for ind,each in enumerate(data):
            path = path+str(ind)
            cv2.imwrite(path+".png", each)
    return files,data

In [31]:
path = 'images/collected/'
fl,data = read_all_images(path, mode=False)

Total_nos_images ==> 21


In [32]:
print(fl)

['paper1.jpg', 'paper2.png', 'paper3.jpg', 'paper4.jpeg', 'paper5.jpg', 'paper7.jpg', 'paper8.jpg', 'paper9.jpg', 'paper10.jpg', 'paper11.jpg', 'paper12.jpg', 'paper13.jpg', 'paper14.jpg', 'paper15.jpg', 'paper16.jpg', 'paper17.jpg', 'paper18.jpg', 'paper19.jpg', 'paper20.jpg', 'paper21.jpg', 'paper22.jpg']


In [33]:
data = data[:2]

In [34]:
def apply_threshold(imgArr, filepath:str, filename:str, params:tuple):
    if not os.path.exists(filepath):
        os.makedirs(filepath)
        print('\nCreating directory ->',filepath)
    
    methods = {'binary': cv2.THRESH_BINARY,
               'binary_inv': cv2.THRESH_BINARY_INV,
               'otsu': cv2.THRESH_OTSU}
    
    copy = imgArr.copy()
    gray = cv2.cvtColor(imgArr, cv2.COLOR_BGR2GRAY) #grayscale conversion

    if params[2] in list(methods.keys()):
        ret,thres = cv2.threshold(gray, params[0], params[1], methods[params[2]])
    else:
        print('Thresholding method not found from ',list(methods.keys()))
    if filename:
        cv2.imwrite(filepath+filename, thres)
    print('\nApplied threshold..')
    
    return thres

In [35]:
def make_denoised(imgArr, filepath:str, filename:str, params:tuple):
    if not os.path.exists(filepath):
        os.makedirs(filepath)
        print('\nCreating directory ->',filepath)
        
    copy = imgArr.copy()
    gray = cv2.cvtColor(imgArr, cv2.COLOR_BGR2GRAY)
    denoised = cv2.fastNlMeansDenoising(imgArr,None,
                                        params[0],params[1],params[2])
    
    if filename:
        cv2.imwrite(filepath+filename, denoised)
    print('\nApplied denoising..')
    return denoised

In [36]:
def apply_morphology(imgArr, filepath:str, filename:str, params:tuple):
    if not os.path.exists(filepath):
        os.makedirs(filepath)
        print('\nCreating directory ->',filepath)
        
    copy = imgArr.copy()
    if params[0] == 'dilation':
        size = params[1]
        kernel = np.ones((size,size), dtype=np.float32)
        dilated = cv2.dilate(imgArr, kernel)
        output = dilated
        print('\nApplied dilation..')
        
    if params[0] == 'erosion':
        size = params[1]
        kernel = np.ones((size,size), dtype=np.float32)
        eroded = cv2.erode(imgArr, kernel)
        output = eroded
        print('\nApplied erosion..')
        
    if filename:   
        cv2.imwrite(filepath+filename, output) 
    
    return output

In [37]:
def perform_RLSA_hor(imgArr, filepath:str, filename:str):
#     try:
    if not os.path.exists(filepath):
        os.makedirs(filepath)
        print('\nCreating directory ->',filepath)

    x, y = imgArr.shape[0],imgArr.shape[1]
    value = max(math.ceil(x/100),math.ceil(y/100))+20

    mask = rlsa.rlsa(imgArr, True, False, value)

    if filename:
        cv2.imwrite(filepath+filename, mask)

    return mask

def perform_RLSA_ver(imgArr, filepath:str, filename:str):
#     try:
    if not os.path.exists(filepath):
        os.makedirs(filepath)
        print('\nCreating directory ->',filepath)

    x, y = imgArr.shape[0],imgArr.shape[1]
    value = max(math.ceil(x/100),math.ceil(y/100))+20

    mask = rlsa.rlsa(imgArr, False, True, value)

    if filename:
        cv2.imwrite(filepath+filename, mask)
        
        return mask

def get_RLSA_final(imgArr1, imgArr2, filepath:str, filename:str):
    try:
        if not os.path.exists(filepath):
            os.makedirs(filepath)
            print('\nCreating directory ->',filepath)

        rlsa_f = cv2.bitwise_or(imgArr1,imgArr2)
        rlsa_f = cv2.bitwise_not(rlsa_f)

        if filename:
            cv2.imwrite(filepath+filename, rlsa_f)

        return rlsa_f
    except:
        print('\nerrorjgv')

In [69]:
def get_segments(imgList, orgList, filepath:str, no_fill = False):
    if not os.path.exists(filepath):
        os.makedirs(filepath)
        print('\nCreating directory ->',filepath)
        
    if no_fill:
        thickness = 1
    else:
        thickness = -1
        
    for ind in range(0,len(data_dil)):
        contours, hierarchy = cv2.findContours(imgList[ind], cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnt = 0

        for cnts in contours:
            peri = cv2.arcLength(cnts, True)
            epsilon = 0.001*peri
            approx = cv2.approxPolyDP(cnts, epsilon, True)
            cnt+=1
            cv2.drawContours(orgList[ind], [approx], -1, (0, 0, 255), thickness)

        cv2.imwrite(filepath+'/final_result_'+str(ind)+'.png', orgList[ind])
        cv2.imshow("show",orgList[ind])
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    return

In [38]:
filepath_1 = "test_images/denoised/"
filepath_2 = "test_images/threshold/"
filepath_3 = "test_images/RLSA_h/"
filepath_4 = "test_images/RLSA_v/"
filepath_5 = "test_images/RLSA_f/"

params_denoise = (10,7,21) # filter strength, templateWindowSize, searchWindowSize
params_thres = (200, 255, 'otsu')

for ind,imgArr in enumerate(data):
    filename_1 = 'denoised_'+str(ind)+".png"
    filename_2 = 'thres_'+str(ind)+".png"
    filename_3 = 'rlsa_h_'+str(ind)+".png"
    filename_4 = 'rlsa_v_'+str(ind)+".png"
    filename_5 = 'rlsa_f_'+str(ind)+".png"
    
    denoised = make_denoised(imgArr,filepath_1,filename_1,params_denoise)
    thres = apply_threshold(imgArr,filepath_2,filename_2,params_thres)


Applied denoising..

Applied threshold..

Applied denoising..

Applied threshold..


In [39]:
filepath_3

'test_images/RLSA_h/'

In [40]:
filepath_2 = "test_images/threshold/"

thres_data = read_all_images(filepath_2, mode='binary')

for ind,imgArr in enumerate(thres_data[1]):
    filename_3 = 'rlsa_h_'+str(ind)+".png"
    rlsa_h = perform_RLSA_hor(imgArr,filepath_3,filename_3)

Total_nos_images ==> 2


In [42]:
filepath_2 = "test_images/threshold/"

thres_data = read_all_images(filepath_2, mode='binary')

for ind,imgArr in enumerate(thres_data[1]):
    filename_4 = 'rlsa_v_'+str(ind)+".png"
    rlsa_v = perform_RLSA_ver(imgArr,filepath_4,filename_4)

Total_nos_images ==> 2


In [43]:
filepath_3 = "test_images/RLSA_h/"
filepath_4 = "test_images/RLSA_v/"
filepath_5 = "test_images/RLSA_f/"

_,data_h = read_all_images(filepath_3,mode=False)
_,data_v = read_all_images(filepath_4,mode=False)

for ind in range(0,len(data_h)):
    filename_5 = 'rlsa_f_'+str(ind)+".png"
    get_RLSA_final(data_h[ind],data_v[ind],filepath_5,filename_5)

Total_nos_images ==> 2
Total_nos_images ==> 2


In [44]:
_,data_rlsa = read_all_images(filepath_5, mode='binary')

for ind in range(0,len(data_rlsa)):
    apply_morphology(data_rlsa[ind], "test_images/dilated/", 'dilated_f_'+str(ind)+".png", ('dilation', 7))

Total_nos_images ==> 2

Applied dilation..

Applied dilation..


In [70]:
_,data_dil = read_all_images("test_images/dilated/", mode='binary')
original = copy.deepcopy(data)

filepath_final = 'test_images/final_result'
get_segments(data_dil, original, filepath_final, no_fill = False)

Total_nos_images ==> 2


In [71]:
_,data_dil = read_all_images("test_images/dilated/", mode='binary')
original = copy.deepcopy(data)

filepath_final = 'test_images/final_result_1'
get_segments(data_dil, original, filepath_final, no_fill = True)

Total_nos_images ==> 2
